In [1]:
import pylangacq as pla

In [2]:
import numpy as np

In [3]:
import pandas as pd

In [4]:
import glob

In [5]:
import os

In [6]:
pla.__version__

'0.11.0'

In [7]:
# data = pla.read_chat('./Pitt/*.cha')

Data grab

In [8]:
cookie_data = pla.read_chat('./Pitt/*/cookie/*.cha')

In [9]:
sentence_data = pla.read_chat('./Pitt/*/sentence/*.cha')

In [10]:
sentence_fnames = glob.glob(os.getcwd()+'/Pitt/*/sentence/*.cha')

In [14]:
sentence_fnames = sorted(sentence_fnames)

In [15]:
sentence_fnames[0]

'/Users/renee/Documents/WIS_Spr20/DL/FinalProj/Pitt/Dementia/sentence/001-0.cha'

In [16]:
# Control, Dementia
cookie_fnames = glob.glob(os.getcwd()+'/Pitt/*/cookie/*.cha')

In [17]:
cookie_fnames = sorted(cookie_fnames)

Data exploration

In [18]:
len(cookie_data)

552

In [19]:
len(sentence_data)

240

In [20]:
cookie_data.number_of_utterances('PAR')

7054

Targets (MMSE scores)

In [21]:
# fill missing mmse with group's mean mmse
def get_targets(dataset,filenames):
    
    targets = []
    groups = {}
    
#     for fname in sorted(filenames):
    for fname in filenames:
        # PAR MMSE
#         print(fname)
        target = dataset.headers()[fname]['Participants']['PAR']['education']
        group = dataset.headers()[fname]['Participants']['PAR']['group']
        
        # to be replaced with group mean
        if target is '':
            targets.append(group)
            
        else:
            targets.append(int(target))
            if group in groups.keys():
                groups[group].append(int(target))
            else:
                groups[group] = [int(target)]
                
#     print('targets',targets)            
#     print('groups',groups)
    
    for g in groups.keys():
        m = np.mean(groups[g])
        groups[g] = int(m) if m % 1 > 5 else int(m) + 1
    
    #TODO replace with tensor
    targetseries = pd.Series(targets)
    
#     print('groups_mean',groups)
    
    for g in groups.keys():
        targetseries[[i for i,x in enumerate(targets) if x==g]] = groups[g]

    return targetseries

In [22]:
sentence_data_targets = get_targets(sentence_data,sentence_fnames)

In [23]:
cookie_data_targets = get_targets(cookie_data,cookie_fnames)

In [19]:
len(sentence_data_targets) == len(sentence_data)

True

In [20]:
len(cookie_data_targets) == len(cookie_data)

True

BUILD VOCAB

In [71]:
vocabset = set()

In [63]:
len(cookie_fnames)

552

In [72]:
for corpus,data in [(cookie_fnames,cookie_data),(sentence_fnames,sentence_data)]: 
    for i in range(len(corpus)):
        tokens = set([tokenraw for sent in data.tagged_sents(participant='PAR',by_files=True)[corpus[i]] for (tokenraw,pos,tokenstem,dependency) in sent])
        vocabset.update(tokens)

In [74]:
len(vocabset)

2188

In [ ]:
for i in range(len(sentence_fnames)):

BUILD MOR: POS + GRAMMATICAL CATEGORIES

In [75]:
posset = set()

In [76]:
for corpus,data in [(cookie_fnames,cookie_data),(sentence_fnames,sentence_data)]: 
    for i in range(len(corpus)):
        pos = set([pos for sent in data.tagged_sents(participant='PAR',by_files=True)[corpus[i]] for (tokenraw,pos,tokenstem,dependency) in sent])
        posset.update(pos)

In [77]:
len(pos)

13

In [78]:
pos

{'',
 '+...',
 '.',
 'ADV',
 'CO',
 'COP',
 'DET:ART',
 'DET:POSS',
 'META',
 'N',
 'PREP',
 'PRO:SUB',
 'V'}

CLEAN POS, VOCAB

In [29]:
cookie_fnames[0]

'/Users/renee/Documents/WIS_Spr20/DL/FinalProj/Pitt/Control/cookie/002-0.cha'

In [28]:
cookie_data.utterances(participant='PAR',by_files=True)[cookie_fnames[0]]

[('PAR', 'the scene is in the kitchen .'),
 ('PAR',
  'the mother is wiping dishes and the water is running on the floor .'),
 ('PAR',
  "a boy is tryin(g) to get cookies out of a jar and he's about to tip over on a stool ."),
 ('PAR', 'the little girl is reacting to his falling .'),
 ('PAR', 'it seems to be summer out .'),
 ('PAR', 'the window is open .'),
 ('PAR', 'the curtains are blowing .'),
 ('PAR', 'it must be a gentle breeze .'),
 ('PAR', "there's grass outside in the garden ."),
 ('PAR', "mother's finished certain of the dishes ."),
 ('PAR', "kitchen's very tidy ."),
 ('PAR',
  'the mother seems to have nothing in the house to eat except cookies in the cookie jar .'),
 ('PAR', 'the children look to be almost about the same size .'),
 ('PAR', "perhaps they're twins ."),
 ('PAR', "they're dressed for summer warm weather ."),
 ('PAR', 'you want more ?'),
 ('PAR', "the mother's in a short sleeve dress ."),
 ('PAR', "I'll hafta say it's warm .")]

In [33]:
#utterances for one PAR - 468-0
sentence_data.utterances(participant='PAR',by_files=True)[sentence_fnames[0]]

[('PAR', 'I wrote with my pencil .'),
 ('PAR', 'the tree is beautiful .'),
 ('PAR', 'the child was taken to the hospital .'),
 ('PAR', "I can't think of any right now ."),
 ('PAR', 'I +...')]

In [58]:
[sent for (partag,sent) in sentence_data.utterances(participant='PAR',by_files=True)[sentence_fnames[1]]]

['I broke my pencil .',
 'the tree is +...',
 'no wait a minute I wanna get something in there .',
 'the tree fell down .',
 'mhm .',
 'the child was sent to the hospital .',
 'cold@q winter@q ?',
 'we had a cold winter in the last winter .',
 'that right ?',
 'our last winter was cold .',
 'chair@q doctor@q and sit@q ?',
 'the doctor sat in his chair .',
 'the +...',
 'I open the bureau drawer .']

In [57]:
sentence_data.tagged_sents(participant='PAR',by_files=True)[sentence_fnames[0]]

[[('I', 'PRO:SUB', 'I', (1, 2, 'SUBJ')),
  ('wrote', 'V', 'write&PAST', (2, 0, 'ROOT')),
  ('with', 'PREP', 'with', (3, 2, 'JCT')),
  ('my', 'DET:POSS', 'my', (4, 5, 'DET')),
  ('pencil', 'N', 'pencil', (5, 3, 'POBJ')),
  ('.', '.', '', (6, 2, 'PUNCT'))],
 [('the', 'DET:ART', 'the', (1, 2, 'DET')),
  ('tree', 'N', 'tree', (2, 3, 'SUBJ')),
  ('is', 'COP', 'be&3S', (3, 0, 'ROOT')),
  ('beautiful', 'ADJ', 'beautiful', (4, 3, 'PRED')),
  ('.', '.', '', (5, 3, 'PUNCT'))],
 [('the', 'DET:ART', 'the', (1, 2, 'DET')),
  ('child', 'N', 'child', (2, 4, 'SUBJ')),
  ('was', 'AUX', 'be&PAST&13S', (3, 4, 'AUX')),
  ('taken', 'PART', 'take&PASTP', (4, 0, 'ROOT')),
  ('to', 'PREP', 'to', (5, 4, 'JCT')),
  ('the', 'DET:ART', 'the', (6, 7, 'DET')),
  ('hospital', 'N', 'hospital', (7, 5, 'POBJ')),
  ('.', '.', '', (8, 4, 'PUNCT'))],
 [('I', 'PRO:SUB', 'I', (1, 4, 'SUBJ')),
  ("can't", 'MOD', 'can', (2, 4, 'AUX')),
  ('CLITIC', 'NEG', 'not', (3, 2, 'NEG')),
  ('think', 'V', 'think', (4, 0, 'ROOT')),
  ('o

In [24]:
pos_test = [tokenstem for sent in cookie_data.tagged_sents(participant='PAR',by_files=True)[cookie_fnames[0]] for (tokenraw,pos,tokenstem,dependency) in sent]

In [25]:
pos_test = ['<EOS>' if tokenstem == '' else tokenstem for tokenstem in pos_test]

In [33]:
vocabset = set()

In [34]:
# lemm
for t in [token.replace('&',' ').replace('-',' ').split(' ')[0] for token in pos_test]:
    vocabset.add(t)

In [49]:
posset = set([pos for sent in cookie_data.tagged_sents(participant='PAR',by_files=True)[cookie_fnames[0]] for (tokenraw,pos,tokenstem,dependency) in sent])

In [50]:
postsettwo = set([pos for sent in cookie_data.tagged_sents(participant='PAR',by_files=True)[cookie_fnames[1]] for (tokenraw,pos,tokenstem,dependency) in sent])

In [51]:
posset

{'.',
 '?',
 'ADJ',
 'ADV',
 'AUX',
 'COORD',
 'COP',
 'DET:ART',
 'DET:POSS',
 'INF',
 'MOD',
 'N',
 'PART',
 'PREP',
 'PRO:EXIST',
 'PRO:INDEF',
 'PRO:PER',
 'PRO:SUB',
 'V'}

In [52]:
postsettwo

{'.',
 'ADJ',
 'ADV',
 'AUX',
 'CM',
 'CO',
 'COORD',
 'COP',
 'DET:ART',
 'DET:NUM',
 'INF',
 'N',
 'PART',
 'PREP',
 'PRO:EXIST',
 'PRO:SUB',
 'QN',
 'V'}

In [42]:
len(postsettwo)

18

In [44]:
len(posset)

19

In [62]:
# this one 
len(postsettwo.union(posset))

23

In [67]:
postsettwo.update(posset)

In [68]:
len(postsettwo)

23

In [59]:
postsettwo & posset

{'.',
 'ADJ',
 'ADV',
 'AUX',
 'COORD',
 'COP',
 'DET:ART',
 'INF',
 'N',
 'PART',
 'PREP',
 'PRO:EXIST',
 'PRO:SUB',
 'V'}

In [54]:
postsettwo ^ posset

{'?', 'CM', 'CO', 'DET:NUM', 'DET:POSS', 'MOD', 'PRO:INDEF', 'PRO:PER', 'QN'}

In [58]:
[p for p in postsettwo if p not in posset ]

['CM', 'DET:NUM', 'CO', 'QN']

In [ ]:
postset

In [ ]:
# addtl tags
[token.replace('&',' ').replace('-',' ').split(' ') for token in pos_test]

In [34]:
cookie_data.tagged_sents(participant='PAR',by_files=True)[cookie_fnames[0]]

[[('the', 'DET:ART', 'the', (1, 2, 'DET')),
  ('scene', 'N', 'scene', (2, 3, 'SUBJ')),
  ('is', 'COP', 'be&3S', (3, 0, 'ROOT')),
  ('in', 'PREP', 'in', (4, 3, 'JCT')),
  ('the', 'DET:ART', 'the', (5, 6, 'DET')),
  ('kitchen', 'N', 'kitchen', (6, 4, 'POBJ')),
  ('.', '.', '', (7, 3, 'PUNCT'))],
 [('the', 'DET:ART', 'the', (1, 2, 'DET')),
  ('mother', 'N', 'mother', (2, 4, 'SUBJ')),
  ('is', 'AUX', 'be&3S', (3, 4, 'AUX')),
  ('wiping', 'PART', 'wipe-PRESP', (4, 0, 'ROOT')),
  ('dishes', 'N', 'dish-PL', (5, 4, 'OBJ')),
  ('and', 'COORD', 'and', (6, 10, 'LINK')),
  ('the', 'DET:ART', 'the', (7, 8, 'DET')),
  ('water', 'N', 'water', (8, 10, 'SUBJ')),
  ('is', 'AUX', 'be&3S', (9, 10, 'AUX')),
  ('running', 'PART', 'run-PRESP', (10, 4, 'CJCT')),
  ('on', 'PREP', 'on', (11, 10, 'JCT')),
  ('the', 'DET:ART', 'the', (12, 13, 'DET')),
  ('floor', 'N', 'floor', (13, 11, 'POBJ')),
  ('.', '.', '', (14, 4, 'PUNCT'))],
 [('a', 'DET:ART', 'a', (1, 2, 'DET')),
  ('boy', 'N', 'boy', (2, 4, 'SUBJ')),
  (

In [59]:
sentence_data.words(participant='PAR',by_files=True)[sentence_fnames[0]]

['I',
 'wrote',
 'with',
 'my',
 'pencil',
 '.',
 'the',
 'tree',
 'is',
 'beautiful',
 '.',
 'the',
 'child',
 'was',
 'taken',
 'to',
 'the',
 'hospital',
 '.',
 'I',
 "can't",
 'think',
 'of',
 'any',
 'right',
 'now',
 '.',
 'I',
 '...']

In [ ]:
sentence

In [39]:
sentence_data.sents(participant='PAR',by_files=True)[sentence_fnames[0]]

[['I', 'wrote', 'with', 'my', 'pencil', '.'],
 ['the', 'tree', 'is', 'beautiful', '.'],
 ['the', 'child', 'was', 'taken', 'to', 'the', 'hospital', '.'],
 ['I', "can't", 'think', 'of', 'any', 'right', 'now', '.'],
 ['I', '...']]

In [40]:
sentence_data.word_frequency(participant='PAR',by_files=True)[sentence_fnames[0]]

Counter({'I': 3,
         'wrote': 1,
         'with': 1,
         'my': 1,
         'pencil': 1,
         '.': 4,
         'the': 3,
         'tree': 1,
         'is': 1,
         'beautiful': 1,
         'child': 1,
         'was': 1,
         'taken': 1,
         'to': 1,
         'hospital': 1,
         "can't": 1,
         'think': 1,
         'of': 1,
         'any': 1,
         'right': 1,
         'now': 1,
         '...': 1})

In [43]:
sentence_data.MLU(participant='PAR')[sentence_fnames[0]]

5.8

In [47]:
#mlu - morphemes
sentence_data.MLUm(participant='PAR')[sentence_fnames[0]]

5.8

In [48]:
# mlu - worrds
sentence_data.MLUw(participant='PAR')[sentence_fnames[0]]

5.0

In [49]:
# index of productive syntax
sentence_data.IPSyn(participant='PAR')[sentence_fnames[0]]

30

In [50]:
#Type token ratio
sentence_data.TTR(participant='PAR')[sentence_fnames[0]]

0.84

In [46]:
sentence_data.part_of_speech_tags(participant='PAR')

{'',
 '!',
 '+"/.',
 '+...',
 '+..?',
 '+/.',
 '+//.',
 '+/?',
 '.',
 '?',
 'ADJ',
 'ADV',
 'ADV:TEM',
 'AUX',
 'BEG',
 'CM',
 'CO',
 'COMP',
 'CONJ',
 'COORD',
 'COP',
 'DET:ART',
 'DET:DEM',
 'DET:NUM',
 'DET:POSS',
 'END',
 'GRAND#N',
 'IN#ADJ',
 'INF',
 'META',
 'MID#N',
 'MOD',
 'N',
 'N:GERUND',
 'N:LET',
 'N:PROP',
 'N:PT',
 'NEG',
 'NEO',
 'PART',
 'POST',
 'PREP',
 'PRO:DEM',
 'PRO:EXIST',
 'PRO:INDEF',
 'PRO:INT',
 'PRO:OBJ',
 'PRO:PER',
 'PRO:POSS',
 'PRO:REFL',
 'PRO:REL',
 'PRO:SUB',
 'QN',
 'UN#ADJ',
 'UN#ADV',
 'V'}

In [18]:
sentence_fnames[:10]

['/Users/renee/Documents/WIS_Spr20/DL/FinalProj/Pitt/Dementia/sentence/468-0.cha',
 '/Users/renee/Documents/WIS_Spr20/DL/FinalProj/Pitt/Dementia/sentence/062-3.cha',
 '/Users/renee/Documents/WIS_Spr20/DL/FinalProj/Pitt/Dementia/sentence/001-0.cha',
 '/Users/renee/Documents/WIS_Spr20/DL/FinalProj/Pitt/Dementia/sentence/674-0.cha',
 '/Users/renee/Documents/WIS_Spr20/DL/FinalProj/Pitt/Dementia/sentence/689-0.cha',
 '/Users/renee/Documents/WIS_Spr20/DL/FinalProj/Pitt/Dementia/sentence/046-2.cha',
 '/Users/renee/Documents/WIS_Spr20/DL/FinalProj/Pitt/Dementia/sentence/711-0.cha',
 '/Users/renee/Documents/WIS_Spr20/DL/FinalProj/Pitt/Dementia/sentence/529-0.cha',
 '/Users/renee/Documents/WIS_Spr20/DL/FinalProj/Pitt/Dementia/sentence/471-0.cha',
 '/Users/renee/Documents/WIS_Spr20/DL/FinalProj/Pitt/Dementia/sentence/319-0.cha']